Arabidopsis seedlings are incubated with certain pharmaceuticals that attack the cell wall of the plants. This leads to increased production of lignin, a polymer that causes the seedlings to "stiffen" so that nothing can get through the cell wall anymore. For analysis, we use a specific dye that docks onto the individual lignin polymers, making them visible. And this is exactly where our interest lies. Using Fiji or ImageJ, we have to manually quantify the stained regions for every sample we analyse, sometimes several hundreds in number. Instead, we would like to have a tool that can be fed with all the images and quantifies the stained regions within seconds.

<img src="images/root0001.png" width="400" height="200">

This cell must be excuted!

First, all required packages for running the notebook are installed. Lines starting with an "#" are comments and will not be executed when running the respective cell.

In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import pandas as pd
import xlsxwriter
import napari

This cell must be excuted!

In [107]:
def save_img(path, img, name):
    '''
    Method for smoothening boundaries in binarized images
    img: image to be saved
    name: name suffix added to original file name
    '''
    txt1 = path.split('.')
    txt1 = f"{txt1[0]}_{name}.tif"
    txt2 = txt1.split('\\')
    txt2 = f"{txt2[0]}_{name}\{txt2[1]}"
    cv.imwrite(txt2, img)

Run the next cell for analysis of all images stored in images/root_images!

In [109]:
# create dataframe
data = {'grayscale intensity stained region':[], 'grayscale intensity cropped region':[], 'grayscale whole image':[], 'area stained region':[], 'area cropped image':[], 'area whole image':[]}
df = pd.DataFrame(data)


# assign directory
directory = 'images/root_images'


# define rgb thresholds
low = np.array([0, 0, 0])
high = np.array([210, 190, 217])


# iterate over files in directory
for filename in os.listdir(directory):
    # path
    f = os.path.join(directory, filename)

    if f.split('.')[1] != 'tif':
        continue
    
    # read image and convert to grayscale
    og_img = cv.imread(f)
    gray_og_img = cv.cvtColor(og_img, cv.COLOR_BGR2GRAY)

    # crop original and grayscale image
    img = og_img[500:-(500+1),500:-(500+1),:]
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # compute mask
    mask = cv.inRange(img, low, high)

    # lay mask over cropped image and cropped grayscale image
    masked_img = cv.bitwise_and(img, img, mask=mask)
    masked_gray_img = cv.bitwise_and(gray_img, gray_img, mask=mask)

    # compute area of stained region, cropped region and whole image
    area_stained = np.sum(masked_gray_img > 0)
    area_cropped = img.shape[0]*img.shape[1]
    area_whole = og_img.shape[0]*og_img.shape[1]

    # compute grayscale intensities
    intensity_stained = np.sum(masked_gray_img) / area_stained
    intensity_cropped = np.sum(masked_gray_img) / area_cropped
    intensity_whole = np.sum(masked_gray_img) / area_whole

    # add new data to dataframe
    new_row = np.array([intensity_stained, intensity_cropped, intensity_whole, area_stained, area_cropped, area_whole])
    df.loc[len(df)] = new_row
    
    # save masked, cropped image
    save_img(f, masked_img, 'msk')


# create excel table from dataframe
df.to_excel("output.xlsx")